In [1]:
import time
import folium
import DBconnection
from PyQt5.QtWidgets import QApplication, QComboBox, QWidget, QVBoxLayout,QGridLayout,QLabel,QRadioButton,QPushButton,QTableWidget, QTableWidgetItem,QLineEdit
from PyQt5.QtWebEngineWidgets import QWebEngineView
from folium.plugins import MarkerCluster
from PyQt5.QtCore import QTimer
cursor, conn = DBconnection.get_connection() 

class Example(QWidget):
    
    def __init__(self):
        super().__init__()
        self.setWindowTitle("UoP (Metro Transit-Minneapolis)")
        self.feature_group = folium.FeatureGroup(name='Vehicle Positions')
        self.setGeometry(300, 300, 900, 700)
        self.central_widget = QWidget(self)
        self.create_map()
        self.initUI()
               

    def initUI(self):
        ###create comboboxes###################
        self.combobox_routes = QComboBox(self)
        self.combobox_trips = QComboBox(self)
        self.combobox_direction = QComboBox(self)
        self.combobox_routes.setFixedWidth(350)
        self.combobox_routes.setFixedHeight(20)
        self.combobox_trips.setFixedWidth(350)
        self.combobox_trips.setFixedHeight(20)
        self.combobox_direction.setFixedWidth(30)
        self.combobox_direction.setFixedHeight(20)        
        #######################################
        
        ###button#################
        #btn = QPushButton("clear RadioButton")
        #######################################
        
        ###create Radio button#################
        self.radioButton_route = QRadioButton('display route-trip', self)
        self.radioButton_stops = QRadioButton('display stops', self)
        #######################################
        
        ###create reset RadioButton button#####
        self.resetRadioButton = QPushButton('Reset')
        #######################################
        
        ##create label#########################
        self.trip_direction = QLabel('Direction:')
        self.num_of_stops_label = QLabel('Number of stops:')
        self.trip_dist_label = QLabel('Trip Length(km):')
        self.traveled_dist_label = QLabel('Traveled Distance(km):')
        self.routes_label = QLabel('Routes:')
        self.trips_label = QLabel('Trips Headsign:')
        self.num_of_trajectories = QLabel('Buses:')
        #######################################
        
        ##create textbox#######################
        self.trip_length = QLineEdit(self)
        self.trip_length.setReadOnly(True)
        self.num_of_stops = QLineEdit(self)
        self.num_of_stops.setReadOnly(True)
        self.dist_traveled = QLineEdit(self)
        self.dist_traveled.setReadOnly(True)
        self.route_alert = QLineEdit(self)
        self.route_alert.setReadOnly(True)
        self.trajectories = QLineEdit(self)
        self.trajectories.setReadOnly(True)        
        #######################################
        
        ###layout##############################
        #layout = QVBoxLayout(self)
        layout = QGridLayout(self)
        layout.addWidget(self.routes_label,0,0)
        
        layout.addWidget(self.combobox_routes,1,0)
        layout.addWidget(self.num_of_stops_label,1,1)
        layout.addWidget(self.num_of_stops,1,2)
        
        layout.addWidget(self.trips_label,2,0)
        layout.addWidget(self.trip_dist_label,2,1)
        layout.addWidget(self.trip_length,2,2)
        
        layout.addWidget(self.combobox_trips,3,0)
        layout.addWidget(self.trip_direction,3,1)
        layout.addWidget(self.combobox_direction,3,2)
        layout.addWidget(self.traveled_dist_label,3,3)
        layout.addWidget(self.dist_traveled,3,4) 
        
        layout.addWidget(self.radioButton_route,4,0)
        layout.addWidget(self.num_of_trajectories,4,1)
        layout.addWidget(self.trajectories,4,2)
        
        layout.addWidget(self.radioButton_stops,5,0)
        #layout.addWidget(self.btn,5,1)
        #layout.addWidget(self.resetRadioButton)

        layout.addWidget(self.route_alert,6,0)
        layout.addWidget(self.map_view,7,0)
        self.setLayout(layout)
        #######################################
        
        ###call functions######################
        self.fill_combobox_routes()
        self.setGeometry(500, 500, 800, 800)
        self.show()
        #######################################
     
    
    
    #add items to combobox_routes
    def fill_combobox_routes(self):
        self.combobox_routes.addItem('')
        self.combobox_routes.addItem('-')
        
        self.combobox_direction.addItem(' ')
        self.combobox_direction.addItem('1')
        self.combobox_direction.addItem('0')
        
        cursor.execute("""select distinct  r.route_desc from routes r
                        where r.feed_version=(select routes.feed_version from routes order by routes.feed_version desc limit 1)""")
        records_routes = cursor.fetchall()
        for routes in records_routes:
            self.combobox_routes.addItem(routes[0])
        #btn.clicked.connect(self.update_map)   
        # connect currentIndexChanged signal of combobox1 to update combobox2
        self.combobox_routes.currentIndexChanged.connect(self.fill_combobox_trips)    

        
    def fill_combobox_trips(self):
        # clear existing items in combobox_trip
        self.combobox_trips.clear()
        self.combobox_trips.addItem('')
        self.combobox_trips.addItem('-')
        # get selected value from combobox_routes
        route = self.combobox_routes.currentText()
        cursor.execute("""select distinct trips.trip_headsign from routes
                          inner join trips
                          on routes.route_id = trips.route_id
                          where routes.route_desc=%s
                          and routes.feed_version=(select routes.feed_version from routes order by routes.feed_version desc limit 1)
                          and trips.feed_version=(select routes.feed_version from routes order by routes.feed_version desc limit 1)""",(route,))
        records_routes = cursor.fetchall()
        for routes in records_routes:
            self.combobox_trips.addItem(routes[0])
       
        self.combobox_trips.currentIndexChanged.connect(self.update_map)
            
        
    def create_map(self):
        self.map = folium.Map(location=[44.965770, -93.285595], zoom_start=10)
        self.map_view = QWebEngineView(self.central_widget)
        self.map_view.setHtml(self.map._repr_html_())
        
        
    def reset_RadioButton(self):
        self.radioButton_route.setChecked(False)
        self.radioButton_stops.setChecked(False)
        
        
    def update_map(self):
            self.timer=QTimer()
            #self.timer.stop()
            ###get values from combobox###################
            route_map = self.combobox_routes.currentText()
            trip_map = self.combobox_trips.currentText()
            direction='-'
            direction = self.combobox_direction.currentText()
            self.combobox_direction.currentIndexChanged.connect(self.update_map) 
            ##############################################

            ###check if radiobutton is clicked############
            self.radioButton_route.clicked.connect(self.update_map)
            self.radioButton_stops.clicked.connect(self.update_map)      
            ##############################################

            ##refresh(recreate) map to 'clean' old map data and display new######
            self.map = folium.Map(location=[44.965770, -93.285595], zoom_start=10)
            self.map_view.setHtml(self.map._repr_html_())
            ##############################################

            ##set textBoxes to empty######################
            self.num_of_stops.setText(' ')
            self.trip_length.setText(' ')
            self.dist_traveled.setText(' ')
            self.route_alert.setText(' ')
            ##############################################

           # print(route_map,'  ',trip_map)
            max_stop_seq_trip_id=None
            ####### display vehicle positions from trip#############################################################
            if ((route_map!= None or route_map != '-') and(trip_map!= None and  trip_map!='-') and (direction !=' ') ):#and not(self.radioButton.isChecked())):
               #while True:
                    
                    cursor.execute("""select distinct 
                                        trajectories.lat,trajectories.lon,
                                        ( select min_dist.seq from (select stop_times.stop_sequence seq,   
                                                        min(ST_Distance(ST_Transform(ST_SetSRID(ST_MakePoint(trajectories.lon,trajectories.lat), 4326), 3857),
                                                        ST_Transform(ST_SetSRID(ST_MakePoint(stops.stop_lon,stops.stop_lat), 4326), 3857))) dist from stop_times,stops
                                                        where trajectories.trip_id = stop_times.trip_id
                                                        and stop_times.stop_id = stops.stop_id
                                                        and stop_times.feed_version=(select routes.feed_version from routes order by routes.feed_version desc limit 1)
                                                        and trajectories.feed_version=(select routes.feed_version from routes order by routes.feed_version desc limit 1)
                                                        group by stop_times.stop_sequence
                                                        order by dist asc
                                                        limit 1) min_dist)min_dist_stop_seq,trajectories.trip_id

                                        from (select vehicle_positions.lat lat, vehicle_positions.lon lon,vehicle_positions.feed_id feed_id ,trips.trip_id trip_id,vehicle_positions.feed_version from routes
                                                                            inner join trips
                                                                            on routes.route_id = trips.route_id
                                                                            and routes.feed_version = trips.feed_version
                                                                            inner join vehicle_positions
                                                                            on trips.trip_id =vehicle_positions.vehicle_trip_trip_id
                                                                            and trips.feed_version = vehicle_positions.feed_version
                                                                            where routes.route_desc=%s
                                                                            and trips.trip_headsign=%s
                                                                            and (vehicle_positions.lon <>0 and vehicle_positions.lat<>0)
                                                                            and vehicle_positions.vehicle_trip_direction_id=%s
                                                                            and routes.feed_version=(select routes.feed_version from routes order by routes.feed_version desc limit 1)
                                                                            order by trips.trip_id) trajectories, 

                                                                            (select max(vehicle_positions.feed_id) feed_id,trips.trip_id trip_id from routes
                                                                            inner join trips
                                                                            on routes.route_id = trips.route_id
                                                                            and trips.feed_version = routes.feed_version
                                                                            inner join vehicle_positions
                                                                            on trips.trip_id =vehicle_positions.vehicle_trip_trip_id
                                                                            and trips.feed_version = vehicle_positions.feed_version
                                                                            where routes.route_desc=%s
                                                                            and trips.trip_headsign=%s
                                                                            and routes.feed_version=(select routes.feed_version from routes order by routes.feed_version desc limit 1)
                                                                            group by trips.trip_id
                                                                            order by trips.trip_id)max_feed_trip_id
                                        where 
                                        trajectories.feed_id = max_feed_trip_id.feed_id
                                        and trajectories.trip_id = max_feed_trip_id.trip_id
                                        and trajectories.feed_id in (select distinct vehicle_positions.feed_id from vehicle_positions
                                                                        order by vehicle_positions.feed_id desc
                                                                        limit 3)
                                        """,(route_map,trip_map,direction,route_map,trip_map,))
                    vehicle_position = cursor.fetchall()
                    
                   # print(route_map,' ',trip_map,' ',direction,route_map,trip_map)
                    #print(vehicle_position)
                    
                    ##route alert###
                    
                    cursor.execute("""SELECT distinct ALERTS.TEXT FROM ALERTS
                                            INNER JOIN ROUTES
                                            ON ALERTS.ROUTE_ID=ROUTES.ROUTE_ID
                                           -- AND ALERTS.FEED_VERSION=ROUTES.FEED_VERSION
                                            WHERE ROUTES.ROUTE_DESC=%s
                                            AND ALERTS.DIRECTION_ID=%s
                                            AND (ALERTS.STOP_ID IS NULL OR ALERTS.STOP_ID='')
                                            --AND ALERTS.FEED_VERSION=(select routes.feed_version from routes order by routes.feed_version desc limit 1)
                                            AND ALERTS.FEED_ID=(SELECT MAX(ALERTS.FEED_ID) FROM ALERTS)
                                    """,(route_map,direction,))
                    route_alert_stops = cursor.fetchall()
                    if len(route_alert_stops)>0:
                        route_alerts_data=[]
                        for alerts in route_alert_stops:
                            route_alerts_data.append(alerts)   
                      
                        for alert in route_alerts_data:
                            alert=str(alert[0])
                            self.route_alert.setText(alert)
                    ##################
                    ''''
                    ########## display trip route (RadioButton) ################################################################ 
                    if  (self.radioButton_route.isChecked()):
                        cursor.execute("""select shapes.shape_pt_lat, shapes.shape_pt_lon from shapes
                                        where shapes.shape_id in (
                                        select trips.shape_id from routes
                                        inner join trips
                                        on routes.route_id = trips.route_id
                                        inner join vehicle_positions
                                        on trips.trip_id =vehicle_positions.vehicle_trip_trip_id
                                        where routes.route_desc=%s
                                        and trips.trip_headsign=%s
                                        order by vehicle_positions.feed_id desc
                                        limit 1)
                                        order by shapes.shape_pt_sequence asc
                                    """,(route_map,trip_map,))
                        trip_route = cursor.fetchall()
                        print('route')
                        if len(trip_route)>0:
                            sql_data=[]#list to store sql recors
                            for position in trip_route:#append data to sql_data
                                sql_data.append(position)
                            self.polyline=folium.PolyLine(locations=sql_data, color='red', weight=5)  
                            self.polyline.add_to(self.map)                    
                         ########################################################################## 
                        '''
                        
                    if len(vehicle_position)>0:
                        sql_data=[]#list to store sql recors
                        for position in vehicle_position:#append data to sql_data
                            sql_data.append(position)
                            self.marker = folium.Marker(location=position[:2],popup=position[-1:], radius=3, color='blue', fill=True, fill_color='red')
                            self.marker.add_to(self.map)

                        num_marker=0
                        
                        for position in sql_data:
                            if position[0] !=0.0 and position[1]!=0.0:
                                num_marker+=1          
                        self.trajectories.setText(str(num_marker))   
                        self.map_view.setHtml(self.map._repr_html_())
                        
                        ##find max stop_seq from trip_id##
                        max_stop_seq=-1
                        for i in sql_data:
                            if  i[2] >max_stop_seq:
                                max_stop_seq=i[2]
                                max_stop_seq_trip_id=i[3]

                        ###############################

                        ####display trip length################
                        if len(max_stop_seq_trip_id)>0:
                            cursor.execute("""SELECT ROUND(CAST(shapes.shape_dist_traveled * 0.0003048 AS numeric), 2) 
                                                FROM shapes
                                                inner join trips
                                                on shapes.shape_id= trips.shape_id
                                                where trips.trip_id in (
                                                select 
                                                vehicle_trip_trip_id as trip_id
                                                from routes
                                                inner join trips
                                                on routes.route_id = trips.route_id
                                                and routes.feed_version = trips.feed_version
                                                inner join vehicle_positions
                                                on trips.trip_id =vehicle_positions.vehicle_trip_trip_id
                                                and trips.feed_version = vehicle_positions.feed_version
                                                where routes.route_desc=%s
                                                and trips.trip_headsign=%s
                                                and vehicle_positions.vehicle_trip_trip_id=%s
                                                and routes.feed_version=(select routes.feed_version from routes order by routes.feed_version desc limit 1)
                                                order by vehicle_positions.feed_id desc
                                                limit 1)
                                                and shapes.feed_version=(select routes.feed_version from routes order by routes.feed_version desc limit 1)
                                                order by shapes.shape_dist_traveled desc
                                                limit 1
                                                """,(route_map,trip_map,max_stop_seq_trip_id))
                            trip_length = cursor.fetchall()
                            if len(trip_length)>0:
                                length=str(trip_length[0][0])
                                self.trip_length.setText(length)

                        #######################################

                        ####display number of stops################ 
                        if len(max_stop_seq_trip_id)>0:
                            cursor.execute("""select count(distinct stops.*) from stop_times
                                                inner join stops
                                                on stop_times.stop_id= stops.stop_id
                                                and stop_times.feed_version=stops.feed_version
                                                where stop_times.trip_id in (
                                                select 
                                                vehicle_trip_trip_id as trip_id
                                                from routes
                                                inner join trips
                                                on routes.route_id = trips.route_id
                                                and routes.feed_version=trips.feed_version
                                                inner join vehicle_positions
                                                on trips.trip_id =vehicle_positions.vehicle_trip_trip_id
                                                and trips.feed_version = vehicle_positions.feed_version
                                                where routes.route_desc=%s
                                                and trips.trip_headsign=%s
                                                and vehicle_positions.vehicle_trip_trip_id=%s
                                                and routes.feed_version=(select routes.feed_version from routes order by routes.feed_version desc limit 1)
                                                order by vehicle_positions.feed_id desc
                                                limit 1)
                                                and stop_times.feed_version=(select routes.feed_version from routes order by routes.feed_version desc limit 1)
                                                """,(route_map,trip_map,max_stop_seq_trip_id))
                            #print(route_map,trip_map,max_stop_seq_trip_id)
                            num_of_stops = cursor.fetchall()
                            if len(num_of_stops)>0:
                                stops=str(num_of_stops[0][0])
                                self.num_of_stops.setText(stops)
                        #######################################

                        ####display distance traveled################
                        if len(max_stop_seq_trip_id)>0:
                            cursor.execute("""select 
                                                    ROUND(CAST(shape_dist_traveled * 0.0003048 AS numeric), 2),
                                                    ST_Distance(
                                                        ST_Transform(ST_SetSRID(ST_MakePoint(veh_position.veh_lon,veh_position.veh_lat), 4326), 3857),
                                                        ST_Transform(ST_SetSRID(ST_MakePoint(shapes.shape_pt_lon,shapes.shape_pt_lat), 4326), 3857)) as distance
                                                    from 
                                                    (
                                                    select 
                                                    vehicle_trip_trip_id as trip_id,
                                                    vehicle_positions.lon as veh_lon,vehicle_positions.lat as veh_lat
                                                    from routes
                                                    inner join trips
                                                    on routes.route_id = trips.route_id
                                                    inner join vehicle_positions
                                                    on trips.trip_id =vehicle_positions.vehicle_trip_trip_id
                                                    where routes.route_desc=%s
                                                    and trips.trip_headsign=%s
                                                    and vehicle_positions.vehicle_trip_trip_id=%s
                                                    order by vehicle_positions.feed_id desc
                                                    limit 1) veh_position,trips, shapes
                                                    where veh_position.trip_id=trips.trip_id
                                                    and shapes.shape_id=trips.shape_id
                                                    order by distance asc
                                                    limit 1
                                                """,(route_map,trip_map,max_stop_seq_trip_id))
                            dist_traveled = cursor.fetchall()
                            if len(dist_traveled)>0:
                                stops=str(dist_traveled[0][0])
                                self.dist_traveled.setText(stops)

                        self.timer.timeout.connect(self.update_map)
                        self.timer.start(17000)
                   # time.sleep(16)
                        #######################################

            ##############################################################################################


            ########## display all vehicle positions from route ############################################
            if (route_map!='-' and trip_map=='-' ):#and not(self.radioButton_route.isChecked()) and not(self.radioButton_stops.isChecked())):
                cursor.execute("""select distinct 
                                    positions_lat_lon.*
                                    from 
                                    (select vehicle_positions.lat lat, vehicle_positions.lon lon,vehicle_positions.feed_id feed_id,trips.trip_id trip_id,trips.trip_headsign from routes
                                                                    inner join trips
                                                                    on routes.route_id = trips.route_id
                                                                    inner join vehicle_positions
                                                                    on trips.trip_id =vehicle_positions.vehicle_trip_trip_id
                                                                    where routes.route_desc=%s
                                                                    and vehicle_positions.feed_id in (select distinct vehicle_positions.feed_id from vehicle_positions
                                                                                                  order by vehicle_positions.feed_id desc
                                                                                                  limit 3)) positions_lat_lon,

                                    (select max(vehicle_positions.feed_id) feed_id,trips.trip_id trip_id from routes
                                                                    inner join trips
                                                                    on routes.route_id = trips.route_id
                                                                    inner join vehicle_positions
                                                                    on trips.trip_id =vehicle_positions.vehicle_trip_trip_id
                                                                    where routes.route_desc=%s
                                                                    and vehicle_positions.feed_id in (select distinct vehicle_positions.feed_id from vehicle_positions
                                                                                                  order by vehicle_positions.feed_id desc
                                                                                                  limit 3)
                                                                    group by trips.trip_id) trip_id_max_feed_id
                                    where 
                                    positions_lat_lon.trip_id = trip_id_max_feed_id.trip_id
                                    and positions_lat_lon.feed_id = trip_id_max_feed_id.feed_id
                                    and (positions_lat_lon.lat <>0 and positions_lat_lon.lon <>0)
                                """,(route_map,route_map,))
               # print(route_map)
                all_vehicle_positions_route = cursor.fetchall()
                #print(route_map)
                #print(all_vehicle_positions_route)
                if len(all_vehicle_positions_route)>0:
                    sql_data=[]#list to store sql recors
                    for position in all_vehicle_positions_route:#append data to sql_data
                        sql_data.append(position)
                        
                    num_marker=0
                    for position in sql_data:
                        if position[0] !=0.0 and position[1]!=0.0:
                            num_marker+=1
                        self.marker = folium.Marker(location=position[:2], popup=position[-1:], radius=3, color='#ff0000', fill=True, fill_color='red')
                        self.marker.add_to(self.map)
                    self.map_view.setHtml(self.map._repr_html_())
                    self.trajectories.setText(str(len(all_vehicle_positions_route)))
                    #self.timer=QTimer()
                    self.timer.timeout.connect(self.update_map)
                    self.timer.start(16000) 
            ##############################################################################################


            ########## display all vehicle positions #######################################################
            if (route_map =='-' and trip_map =='-'): #and not(self.radioButton_route.isChecked()) and not(self.radioButton_stops.isChecked())):
                cursor.execute("""select distinct vehicle_positions.lat,vehicle_positions.lon,vehicle_positions.feed_id,routes.route_desc from 
                                    (select distinct routes.route_id route,trips.trip_id trip,max(vehicle_positions.feed_id) feed
                                        from routes
                                                                    inner join trips
                                                                    on routes.route_id = trips.route_id
                                                                    inner join vehicle_positions
                                                                    on trips.trip_id =vehicle_positions.vehicle_trip_trip_id
                                                                    where vehicle_positions.feed_id in (select distinct vehicle_positions.feed_id from vehicle_positions
                                                                                                        order by vehicle_positions.feed_id desc
                                                                                                        limit 3)
                                                                    and vehicle_positions.lat<>0
                                                                    group by routes.route_id,trips.trip_id)tbla,vehicle_positions,routes


                                    where  vehicle_positions.vehicle_trip_trip_id = tbla.trip
                                    and vehicle_positions.vehicle_trip_route_id = tbla.route
                                    and vehicle_positions.feed_id = tbla.feed
                                    and routes.route_id=vehicle_positions.vehicle_trip_route_id
                                    and (vehicle_positions.lat<>0 and vehicle_positions.lon<>0)""")
                all_vehicle_positions = cursor.fetchall()
                
                if len(all_vehicle_positions)>0:
                    sql_data=[]#list to store sql recors
                    for position in all_vehicle_positions:#append data to sql_data
                        sql_data.append(position)
                        
                    num_marker=0
                    for position in sql_data:
                        if position[0] !=0.0 and position[1]!=0.0:
                            num_marker+=1 
                        self.marker = folium.Marker(location=position[:2], popup=position[-1:], radius=3, color='blue', fill=True, fill_color='red')
                        self.marker.add_to(self.map)
                    
                    self.trajectories.setText(str(num_marker))
                    self.map_view.setHtml(self.map._repr_html_())
                    self.timer.timeout.connect(self.update_map)
                    self.timer.start(16000) 
            ##############################################################################################   

            
            ########## display trip route (RadioButton) ################################################################ 
            if ((route_map!= None or route_map != '-')and(trip_map!= None) and  (trip_map!='-') and (self.radioButton_route.isChecked())):
                    cursor.execute("""select shapes.shape_pt_lat, shapes.shape_pt_lon from shapes
                                    where shapes.shape_id in (
                                    select trips.shape_id from routes
                                    inner join trips
                                    on routes.route_id = trips.route_id
                                    inner join vehicle_positions
                                    on trips.trip_id =vehicle_positions.vehicle_trip_trip_id
                                    where routes.route_desc=%s
                                    and trips.trip_headsign=%s
                                    order by vehicle_positions.feed_id desc
                                    limit 1)
                                    order by shapes.shape_pt_sequence asc
                                """,(route_map,trip_map,))
                    trip_route = cursor.fetchall()
                    if len(trip_route)>0:
                        sql_data=[]#list to store sql recors
                        for position in trip_route:#append data to sql_data
                            sql_data.append(position)
                        
                        
                        self.polyline=folium.PolyLine(locations=sql_data, color='red', weight=5)  
                        self.polyline.add_to(self.map)
                        self.map_view.setHtml(self.map._repr_html_())

            ##############################################################################################
            

            ########### display stops (RadioButton) ####################################################################        
            if  (self.radioButton_stops.isChecked()):
                    cursor.execute("""select 
                                        stops.stop_name,stops.stop_id,stop_times.stop_sequence,veh_position.trip_id,
                                        ST_Distance(
                                            ST_Transform(ST_SetSRID(ST_MakePoint(veh_position.veh_lon,veh_position.veh_lat), 4326), 3857),
                                            ST_Transform(ST_SetSRID(ST_MakePoint(stops.stop_lon,stops.stop_lat), 4326), 3857)) as distance
                                        from 
                                        (
                                        select 
                                        vehicle_trip_trip_id as trip_id,
                                        vehicle_positions.lon as veh_lon,vehicle_positions.lat as veh_lat
                                        from routes
                                                                        inner join trips
                                                                        on routes.route_id = trips.route_id
                                                                        inner join vehicle_positions
                                                                        on trips.trip_id =vehicle_positions.vehicle_trip_trip_id
                                                                        where routes.route_desc=%s
                                                                        and trips.trip_headsign=%s
                                                                        and vehicle_positions.vehicle_trip_trip_id=%s
                                                                        order by vehicle_positions.feed_id desc
                                                                        limit 1) veh_position,stop_times, stops
                                        where veh_position.trip_id=stop_times.trip_id
                                        and stops.stop_id=stop_times.stop_id
                                        order by distance asc
                                        limit 1
                                """,(route_map,trip_map,max_stop_seq_trip_id))
                    closest_stop = cursor.fetchall()
                    trip_id=0
                    if len(closest_stop)>0:
                        stop_seq=closest_stop[0][2]
                        trip_id=closest_stop[0][3]

                    ##########remaining stops##########
                    if len(trip_id)>0:
                        cursor.execute("""select stops.stop_lat, stops.stop_lon,stops.stop_name,stops.stop_id,stops.stop_name from stop_times
                                            inner join stops
                                            on stop_times.stop_id=stops.stop_id
                                            where stop_times.trip_id=%s
                                            and stop_times.stop_sequence >= %s
                                    """,(max_stop_seq_trip_id,stop_seq,))

                        remaining_stops = cursor.fetchall()

                        sql_data=[]#list to store sql recors
                        for position in remaining_stops:#append data to sql_data
                            sql_data.append(position)

                        if len(remaining_stops)>0:
                            for position in sql_data:
                                self.marker = folium.CircleMarker(location=position[:2], popup=position[-1:], radius=3, color='blue', fill=True, fill_color='red')
                                self.marker.add_to(self.map)

                    #####alerts######  
                    
                    cursor.execute("""SELECT DISTINCT ALERTS.ID,ALERTS.TEXT,ALERTS.STOP_ID FROM ALERTS
                                        INNER JOIN ROUTES
                                        ON ALERTS.ROUTE_ID=ROUTES.ROUTE_ID
                                        WHERE ROUTES.ROUTE_DESC=%s
                                        AND ALERTS.STOP_ID IS NOT NULL
                                """,(route_map,))
                    alert_stops = cursor.fetchall()
                  
                    if len(alert_stops)>0:
                        alerts_data=[]
                        for alerts in alert_stops:
                            alerts_data.append(alerts)   
                        for alert in alerts_data:
                            for position in sql_data:
                                   # print(str(alert),'   ',str(position))
                                    if str(alert[-1]) == str(position[-2]):
                                        self.marker = folium.CircleMarker(location=position[:2], popup=alert[-2:], radius=3, color='red', fill=True, fill_color='red')
                                        self.marker.add_to(self.map)
                        #print('stops 5')
                    ########################################

                    ######pased stops######
                    cursor.execute("""select stops.stop_lat, stops.stop_lon,stops.stop_name,stops.stop_id,stops.stop_name from stop_times
                                        inner join stops
                                        on stop_times.stop_id=stops.stop_id
                                        where stop_times.trip_id=%s
                                        and stop_times.stop_sequence < %s
                                """,(max_stop_seq_trip_id,stop_seq,))

                    traveled_stops = cursor.fetchall()
                   # print(max_stop_seq_trip_id)
                    sql_data=[]#list to store sql recors
                    for position in traveled_stops:#append data to sql_data
                        sql_data.append(position)
                    if len(traveled_stops)>0:
                        for position in sql_data:

                            self.marker = folium.CircleMarker(location=position[:2], popup=position[-1:], radius=3, color='blue', opacity=0.1,fill=True, fill_color='red')
                            self.marker.add_to(self.map)
                    ##alerts##
                    '''
                    for alert in alerts_data:
                            for position in sql_data:
                                    if str(alert[-1]) == str(position[-1]):  
                                        self.marker = folium.CircleMarker(location=position[:2], popup=alert[-2:], radius=3, color='red', fill=True, fill_color='red')
                                        self.marker.add_to(self.map)'''                                      
                    self.map_view.setHtml(self.map._repr_html_()) 
                    #time.sleep(5)
        ##############################################################################################
        
        

if __name__ == '__main__':
    app = QApplication([])
    ex = Example()
    app.exec_()


TypeError: object of type 'int' has no len()

TypeError: object of type 'int' has no len()

TypeError: object of type 'int' has no len()

TypeError: object of type 'int' has no len()